In [1]:
!pip install -q transformers datasets librosa soundfile jiwer accelerate evaluate peft
!pip install -q bitsandbytes  # For 8-bit training if needed
!pip install -q av ffmpeg-python  # For audio decoding support
!apt-get -qq install -y ffmpeg  # System-level ffmpeg

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 37.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 44.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 82.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 68.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━

In [2]:
import os
import pandas as pd
import numpy as np
import torch
import librosa
import soundfile as sf
from pathlib import Path
from tqdm import tqdm
from dataclasses import dataclass
from typing import Any, Dict, List, Union
import re

from transformers import (
    WhisperProcessor,
    WhisperForConditionalGeneration,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
)
from datasets import Dataset, Audio
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import evaluate

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")
print(f"CUDA Version: {torch.version.cuda if torch.cuda.is_available() else 'N/A'}")

# Set paths
TRAIN_PATH = "/kaggle/input/shobdotori/Train"
TRAIN_ANNOTATION_PATH = "/kaggle/input/shobdotori/Train_annotation"
TEST_PATH = "/kaggle/input/shobdotori/Test"
OUTPUT_DIR = "./whisper-bengali-lora"
SUBMISSION_FILE = "submission.csv"

2025-11-15 17:26:47.937125: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763227608.124172     107 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763227608.176404     107 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

Using device: cuda
GPU: Tesla P100-PCIE-16GB
CUDA Version: 12.4


In [3]:
def load_training_data():
    """Load all training data from regional folders and their annotations"""
    all_data = []

    # Get all region folders
    region_folders = [f for f in os.listdir(TRAIN_PATH) if os.path.isdir(os.path.join(TRAIN_PATH, f))]

    print(f"Found {len(region_folders)} regions")

    for region in tqdm(region_folders, desc="Loading regions"):
        # Load annotation CSV for this region
        annotation_file = os.path.join(TRAIN_ANNOTATION_PATH, f"{region}.csv")

        if not os.path.exists(annotation_file):
            print(f"Warning: No annotation file for {region}")
            continue

        # Read annotations
        annotations_df = pd.read_csv(annotation_file)

        # Strip whitespace from column names
        annotations_df.columns = annotations_df.columns.str.strip()

        # Get audio folder path
        audio_folder = os.path.join(TRAIN_PATH, region)

        # Process each annotation
        for idx, row in annotations_df.iterrows():
            audio_file = row['audio'].strip() if 'audio' in annotations_df.columns else row.iloc[0]
            text = row['text'].strip() if 'text' in annotations_df.columns else row.iloc[1]

            audio_path = os.path.join(audio_folder, audio_file)

            if os.path.exists(audio_path):
                all_data.append({
                    'audio': audio_path,
                    'text': text,
                    'region': region
                })

    print(f"\nTotal training samples: {len(all_data)}")
    return pd.DataFrame(all_data)

# Load training data
train_df = load_training_data()
print(f"\nSample data:")
print(train_df.head())

Found 20 regions


Loading regions: 100%|██████████| 20/20 [00:04<00:00,  4.01it/s]


Total training samples: 3350

Sample data:
                                               audio  \
0  /kaggle/input/shobdotori/Train/Mymensingh/fema...   
1  /kaggle/input/shobdotori/Train/Mymensingh/fema...   
2  /kaggle/input/shobdotori/Train/Mymensingh/fema...   
3  /kaggle/input/shobdotori/Train/Mymensingh/fema...   
4  /kaggle/input/shobdotori/Train/Mymensingh/male...   

                              text      region  
0  আজ সকালে আমি বাজারে গিয়েছিলাম।  Mymensingh  
1       তুমি কি নতুন বই পড়তে চাও?  Mymensingh  
2            আকাশে আজ মেঘ জমে আছে।  Mymensingh  
3   আমি বন্ধুদের সাথে খেলা খেলেছি।  Mymensingh  
4   দরজাটা ধীরে ধীরে বন্ধ করে দাও।  Mymensingh  


In [4]:
MODEL_NAME = "openai/whisper-large-v2"

print(f"Loading model: {MODEL_NAME}")
print("This may take a few minutes...")

# Load processor
processor = WhisperProcessor.from_pretrained(
    MODEL_NAME,
    language="bengali",
    task="transcribe"
)


model = WhisperForConditionalGeneration.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float32,
    low_cpu_mem_usage=True
)

# Prepare model for training
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []
model.config.use_cache = False

# Enable gradient checkpointing for memory efficiency
model.gradient_checkpointing_enable()

# Apply LoRA - Target only attention layers
lora_config = LoraConfig(
    r=16,  # REDUCED from 32 to save memory
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],  # Only attention
    lora_dropout=0.05,
    bias="none",
)

# Apply LoRA to model.model (inner Whisper model)
model.model = get_peft_model(model.model, lora_config)
model = model.to(device)

trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model.parameters())
print(f"\n✓ LoRA applied successfully")
print(f"  Trainable params: {trainable_params:,} ({100 * trainable_params / total_params:.2f}%)")
print(f"  Total params: {total_params:,}")

Loading model: openai/whisper-large-v2
This may take a few minutes...


preprocessor_config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/6.17G [00:00<?, ?B/s]

generation_config.json: 0.00B [00:00, ?B/s]


✓ LoRA applied successfully
  Trainable params: 7,864,320 (0.51%)
  Total params: 1,551,169,280


In [5]:
def prepare_dataset(df, processor, test_size=0.1):
    """Prepare dataset for training"""

    def prepare_dataset_batch(batch):
        """Process a single batch"""
        try:
            audio_path = batch["audio"]
            audio_array, sampling_rate = librosa.load(audio_path, sr=16000, mono=True)

            # Compute input features
            input_features = processor.feature_extractor(
                audio_array,
                sampling_rate=16000
            ).input_features[0]

            # Encode target text
            labels = processor.tokenizer(batch["text"]).input_ids

            return {
                "input_features": input_features,
                "labels": labels
            }
        except Exception as e:
            print(f"Error processing {batch['audio']}: {e}")
            return {
                "input_features": np.zeros((80, 3000)),
                "labels": [processor.tokenizer.pad_token_id]
            }

    dataset = Dataset.from_pandas(df[['audio', 'text']])
    dataset = dataset.train_test_split(test_size=test_size, seed=42)

    print("Processing training dataset...")
    train_dataset = dataset["train"].map(
        prepare_dataset_batch,
        remove_columns=dataset["train"].column_names,
        desc="Processing train",
    )

    print("Processing validation dataset...")
    val_dataset = dataset["test"].map(
        prepare_dataset_batch,
        remove_columns=dataset["test"].column_names,
        desc="Processing validation",
    )

    return {"train": train_dataset, "test": val_dataset}

print("Preparing dataset...")
dataset = prepare_dataset(train_df, processor)
print(f"\nTrain samples: {len(dataset['train'])}")
print(f"Validation samples: {len(dataset['test'])}")

Preparing dataset...
Processing training dataset...


Processing train:   0%|          | 0/3015 [00:00<?, ? examples/s]

Processing validation dataset...


Processing validation:   0%|          | 0/335 [00:00<?, ? examples/s]


Train samples: 3015
Validation samples: 335


In [6]:
@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        labels = labels_batch["input_ids"].masked_fill(
            labels_batch.attention_mask.ne(1), -100
        )

        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels
        return batch

data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)
print("✓ Data collator initialized")

✓ Data collator initialized


In [7]:
metric = evaluate.load("wer")

def compute_metrics(pred):
    """Compute WER metric"""
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)
    return {"wer": wer}

print("✓ Metrics function initialized")

✓ Metrics function initialized


In [11]:
training_args = Seq2SeqTrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=4,  # REDUCED from 8
    gradient_accumulation_steps=16,  # INCREASED from 2
    learning_rate=5e-5,  # Higher LR for faster convergence
    warmup_steps=100,  # REDUCED
    num_train_epochs=3,  # REDUCED from 5 for faster training
    gradient_checkpointing=True,  # ENABLED
    fp16=True,  # DISABLED - use FP32 for training stability
    eval_strategy="steps",
    per_device_eval_batch_size=4,  # REDUCED
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=500,
    eval_steps=500,
    logging_steps=25,
    logging_first_step=True,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=False,
    save_total_limit=1,  # REDUCED to save space
    dataloader_num_workers=2,
    remove_unused_columns=False,
    label_names=["labels"],
    optim="adamw_torch",  # Explicit optimizer
)

print("✓ Training arguments configured")

PyTorch: setting up devices


✓ Training arguments configured


In [12]:
trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    processing_class=processor,
)

print("✓ Trainer initialized successfully")
print(f"  Effective batch size: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")
print(f"  Total training steps: ~{len(dataset['train']) * training_args.num_train_epochs // (training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps)}")

Using auto half precision backend


✓ Trainer initialized successfully
  Effective batch size: 64
  Total training steps: ~141


In [13]:
import sys
import gc
import torch
from transformers import logging as hf_logging

hf_logging.set_verbosity_info()

print("\n" + "="*60)
print("STARTING TRAINING")
print("="*60)

try:
    sys.stdout.flush()

    # Clear cache before training (only if CUDA is available)
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    gc.collect()

    # Train the model
    train_result = trainer.train()

    print("\n" + "="*60)
    print("✓ TRAINING COMPLETED!")
    print("="*60)
    # Some Trainer versions use train_result.training_loss; fallback to metrics if needed
    tr_loss = getattr(train_result, "training_loss", None)
    if tr_loss is None and hasattr(train_result, "metrics"):
        tr_loss = train_result.metrics.get("train_loss", None)
    if tr_loss is not None:
        print(f"Training loss: {tr_loss:.4f}")

    # Save the model
    print("\nSaving model...")
    trainer.save_model(OUTPUT_DIR)
    processor.save_pretrained(OUTPUT_DIR)
    print(f"✓ Model and processor saved to {OUTPUT_DIR}")

except Exception as e:
    print("\n" + "="*60)
    print(" TRAINING ERROR")
    print("="*60)
    print(f"Error: {str(e)}") 
    import traceback
    traceback.print_exc()
    raise




STARTING TRAINING


***** Running training *****
  Num examples = 3,015
  Num Epochs = 3
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 16
  Total optimization steps = 144
  Number of trainable parameters = 7,864,320
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Step,Training Loss,Validation Loss


Saving model checkpoint to ./whisper-bengali-lora/checkpoint-144
/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3685: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 448, 'suppress_tokens': [], 'begin_suppress_tokens': [220, 50257]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
Configuration saved in ./whisper-bengali-lora/checkpoint-144/config.json
Configuration saved in ./whisper-bengali-lora/checkpoint-144/generation_config.json
The model is bigger than the maximum size per checkpoint (5GB) and is going to be split in 2 checkpoint shards. You can find where each parameters has been saved in the index located at ./whisper-bengali-lora/checkpoint-144/model.safetensors.index.json.
Feature extractor saved in ./whisper-bengali-lora/checkpoint-144/preprocessor_config.json
tokenizer config file saved in ./whisper-b


✓ TRAINING COMPLETED!
Training loss: 1.3352

Saving model...


The model is bigger than the maximum size per checkpoint (5GB) and is going to be split in 2 checkpoint shards. You can find where each parameters has been saved in the index located at ./whisper-bengali-lora/model.safetensors.index.json.
Feature extractor saved in ./whisper-bengali-lora/preprocessor_config.json
tokenizer config file saved in ./whisper-bengali-lora/tokenizer_config.json
Special tokens file saved in ./whisper-bengali-lora/special_tokens_map.json
added tokens file saved in ./whisper-bengali-lora/added_tokens.json
Feature extractor saved in ./whisper-bengali-lora/preprocessor_config.json
tokenizer config file saved in ./whisper-bengali-lora/tokenizer_config.json
Special tokens file saved in ./whisper-bengali-lora/special_tokens_map.json
added tokens file saved in ./whisper-bengali-lora/added_tokens.json


✓ Model and processor saved to ./whisper-bengali-lora


In [14]:
def clean_bengali_text(text):
    """Clean and normalize Bengali text"""  
    text = re.sub(r'\s+', ' ', text) 
    text = text.strip() 
    text = text.replace('।।', '।') 
    return text       

def transcribe_audio_enhanced(audio_path, model, processor, device): 
    """FIXED: Enhanced transcription with proper dtype handling"""  
    try:
        # Load and preprocess audio
        audio_input, sr = librosa.load(audio_path, sr=16000, mono=True)
        audio_input, _ = librosa.effects.trim(audio_input, top_db=20)
        audio_input = audio_input / (np.max(np.abs(audio_input)) + 1e-8)

        # Process - returns dict with input_features
        inputs = processor(
            audio_input,
            sampling_rate=16000,
            return_tensors="pt"
        )

        input_features = inputs.input_features.to(device)
        if model.dtype == torch.float16:
            input_features = input_features.half()
        elif model.dtype == torch.float32:
            input_features = input_features.float()

        # Set forced decoder IDs for Bengali
        forced_decoder_ids = processor.get_decoder_prompt_ids(language="bengali", task="transcribe")

        # Generate with proper settings
        with torch.no_grad():
            predicted_ids = model.generate(
                input_features,
                forced_decoder_ids=forced_decoder_ids,
                max_length=225,
                num_beams=5,
                early_stopping=True,
                no_repeat_ngram_size=3,
                temperature=0.0,  # Deterministic
            )

        transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]
        return clean_bengali_text(transcription) if transcription else "আমি ভাত খাই।"

    except Exception as e:
        print(f"Error processing {audio_path}: {e}")
        return "আমি ভাত খাই।"

print("✓ Transcription function ready")

✓ Transcription function ready


In [15]:
#### print("\n" + "="*60)
print("PREPARING FOR INFERENCE")
print("="*60)

model = model.to(torch.float16)
print("✓ Model converted to FP16 for inference")

# Clear cache
torch.cuda.empty_cache()
gc.collect()

print("\n" + "="*60)
print("GENERATING PREDICTIONS FOR TEST SET")
print("="*60)

test_files = sorted([f for f in os.listdir(TEST_PATH) if f.endswith('.wav')])
print(f"\nFound {len(test_files)} test files to process")

if len(test_files) == 0:
    print("No test files found!")
else:
    results = []
    model.eval()

    print("\nProcessing test files...")
    for test_file in tqdm(test_files, desc="Transcribing audio"):
        audio_path = os.path.join(TEST_PATH, test_file)
        transcription = transcribe_audio_enhanced(audio_path, model, processor, device)
        results.append({'audio': test_file, 'text': transcription})

    # Create submission dataframe
    submission_df = pd.DataFrame(results)
    submission_df.to_csv(SUBMISSION_FILE, index=False, encoding='utf-8')

    print(f"\n{'='*60}")
    print(f"✓ SUBMISSION FILE CREATED: {SUBMISSION_FILE}")
    print(f"{'='*60}")
    print(f"Total predictions: {len(submission_df)}")
    print(f"\nSample predictions (first 10):")
    print(submission_df.head(10).to_string(index=False))

    # Statistics
    fallback_count = (submission_df['text'] == "আমি ভাত খাই।").sum()
    print(f"\n{'='*60}")
    print(f"Statistics:")
    print(f"  Total files: {len(submission_df)}")
    print(f"  Successful: {len(submission_df) - fallback_count}")
    print(f"  Fallback: {fallback_count}")
    print(f"  Success rate: {100 * (len(submission_df) - fallback_count) / len(submission_df):.1f}%")

    # Text length statistics
    submission_df['text_length'] = submission_df['text'].str.len()
    print(f"\nText length statistics:")
    print(f"  Mean: {submission_df['text_length'].mean():.1f} characters")
    print(f"  Min: {submission_df['text_length'].min()}")
    print(f"  Max: {submission_df['text_length'].max()}")
    print(f"{'='*60}")
    print("✓ Ready for submission!")

PREPARING FOR INFERENCE
✓ Model converted to FP16 for inference

GENERATING PREDICTIONS FOR TEST SET

Found 450 test files to process

Processing test files...


Transcribing audio:   0%|          | 0/450 [00:00<?, ?it/s]Using custom `forced_decoder_ids` from the (generation) config. This is deprecated in favor of the `task` and `language` flags/config options.
Increase max_length from 225 to 228 since input is conditioned on previous segment.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Transcribing audio: 100%|██████████| 450/450 [39:35<00:00,  5.28s/it]


✓ SUBMISSION FILE CREATED: submission.csv
Total predictions: 450

Sample predictions (first 10):
       audio                               text
test_001.wav               তুমে কি খাল বার আপি?
test_002.wav             তুমি আমরে কলোম দিভানি।
test_003.wav          আইস্কের দুপুরি বানিজমিরো।
test_004.wav    আজ কে সবকাল এ হরটা ভিষ্ট নবুলো।
test_005.wav                    তুকি ফারু দুবি?
test_006.wav                   পেকি পন্ধর বিনি?
test_007.wav             আজ কে দোর মিক জুন্সির।
test_008.wav               আর সুর বাইস কুলেজার।
test_009.wav              আমি কান নিক্ষেয় এছো।
test_010.wav তুমে কি আজ বন্দুর সাথা দিখা हরছির?

Statistics:
  Total files: 450
  Successful: 450
  Fallback: 0
  Success rate: 100.0%

Text length statistics:
  Mean: 27.2 characters
  Min: 14
  Max: 52
✓ Ready for submission!
